# Functions

In [ ]:
from pytubefix import YouTube
import whisper
from datetime import timedelta
import google.generativeai as genai
import os
import re
import json
from docx import Document

In [ ]:
def download_audio(url):
    yt = YouTube(url)
    yt.streams.get_audio_only().download(filename="Audio.mp3")

In [ ]:
def format_time(seconds):
    return str(timedelta(seconds=round(seconds, 2)))

def transcription(path_to_audio = "Audio.mp3"):
    model = whisper.load_model("tiny")
    result = model.transcribe("Audio.mp3")
    data = ""
    for i in range(len(result["segments"])):
        text = result["segments"][i]["text"]
        start_time = format_time(result["segments"][i]['start'])
        end_time = format_time(result["segments"][i]['end'])
        
        data += f'{text} Start Time: {start_time} End Time: {end_time}\n'

    return(data)

In [ ]:
def generate_json(data):
    prompt = (
    "You are an expert in creating structured JSON responses from video transcripts to be converted into detailed Word documents. "
    "Format the JSON to include a 'title' field, followed by a 'table_of_contents' field as an array listing the main sections. "
    "Then use a 'sections' field as an array, where each object includes:\n"
    "1. A 'heading' field for the main section title,\n"
    "2. A 'content' field with relevant quotes and timestamps (e.g., [00:05:12]) based on the transcript content. Skip any sponsored segments and focus only on the main content of the video.\n\n"
    "Format Example:\n"
    "{\n"
    "    'title': 'Document Title',\n"
    "    'table_of_contents': ['Section 1', 'Section 2'],\n"
    "    'sections': [\n"
    "        {\n"
    "            'heading': 'Section 1',\n"
    "            'content': '...'\n"
    "        },\n"
    "        {\n"
    "            'heading': 'Section 2',\n"
    "            'content': '...'\n"
    "        }\n"
    "    ]\n"
    "}\n\n"
    f"Video Transcript: {data}\n"
    "JSON Output:"
    )
    genai.configure(api_key=os.environ["GEMINI_API_KEY"])
    model = genai.GenerativeModel(model_name="gemini-1.5-pro-002")
    #print(model.count_tokens(prompt))
    response = model.generate_content(prompt)
    cleaned_content = re.sub(r"```json\s*|\s*```", "", response.text, flags=re.DOTALL).strip()
    return(cleaned_content)    

In [ ]:
def generate_document(JSON):
    data = JSON
    data = json.loads(data)
    doc = Document()
    doc.add_heading(data['title'], level=1)
    doc.add_heading("Table of Contents", level=2)
    for i, toc_item in enumerate(data['table_of_contents'], start=1):
        doc.add_paragraph(f"{i}. {toc_item}")
    for section in data['sections']:
        doc.add_heading(section['heading'], level=2) 
        doc.add_paragraph(section['content'])
    doc_title = data['title'].replace(" ", "_") + ".docx"
    doc.save(doc_title)
    print(f"Document '{doc_title}' created successfully!")

In [ ]:
def final(yt_url):
    download_audio(yt_url)
    generate_document(generate_json(transcription()))